In [1]:
import torch
import torchaudio
from torchaudio.pipelines import MMS_FA as bundle
from typing import List
import IPython
import matplotlib.pyplot as plt
from pypinyin import pinyin, lazy_pinyin, Style

print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

2.1.2+cu118
2.1.2+cu118
cuda


In [2]:
import whisper
from pydub import AudioSegment

In [16]:
model = whisper.load_model("large")
# 使用Whisper進行語音識別
result = model.transcribe("papaya_1.wav")
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 2.0,
  'text': 'Hi 大家好',
  'tokens': [50365, 17155, 220, 15248, 50465],
  'temperature': 0.0,
  'avg_logprob': -0.16101390085402567,
  'compression_ratio': 1.2598684210526316,
  'no_speech_prob': 0.22558216750621796},
 {'id': 1,
  'seek': 0,
  'start': 2.0,
  'end': 5.24,
  'text': '今天我們來聊聊 Python 的 Module 模組',
  'tokens': [50465,
   12074,
   5884,
   3763,
   40096,
   40096,
   15329,
   27949,
   48251,
   220,
   41908,
   30052,
   50627],
  'temperature': 0.0,
  'avg_logprob': -0.16101390085402567,
  'compression_ratio': 1.2598684210526316,
  'no_speech_prob': 0.22558216750621796},
 {'id': 2,
  'seek': 0,
  'start': 5.24,
  'end': 7.7,
  'text': '當我們撰寫 Python 的程式碼時',
  'tokens': [50627,
   13118,
   5884,
   20559,
   108,
   4510,
   104,
   15329,
   27949,
   29649,
   27584,
   16337,
   120,
   6611,
   50750],
  'temperature': 0.0,
  'avg_logprob': -0.16101390085402567,
  'compression_ratio': 1.2598684210526316,
  'no_speech

In [25]:
sentence =[]
for i in range(len(result['segments'])):
    sentence.append([result['segments'][i]['text'].lower(), result['segments'][i]['start'], result['segments'][i]['end']])

sentence

[['hi 大家好', 0.0, 2.0],
 ['今天我們來聊聊 python 的 module 模組', 2.0, 5.24],
 ['當我們撰寫 python 的程式碼時', 5.24, 7.7],
 ['隨著程式碼越來越多', 7.7, 9.74],
 ['結構越變越複雜', 9.74, 11.6],
 ['為了方便管理', 11.6, 13.0],
 ['我們就可以考慮把程式碼一期功能', 13.0, 15.64],
 ['拍成幾個不同的檔案', 15.64, 17.64],
 ['而每個檔案就被稱為是一個模組', 17.64, 20.7],
 ['那在 pycharm 建立模組非常的簡單', 20.7, 23.64],
 ['我們只要到左欄打開右鍵選單', 23.64, 26.54],
 ['點擊新增', 26.54, 28.16],
 ['pycharm 檔案', 28.16, 29.64],
 ['然後替這個模組取一個名字', 29.64, 32.04],
 ['譬如我的模組', 32.04, 33.84],
 ['pycharm 就會自動幫我們打開一個新的分頁', 33.84, 37.44],
 ['這時我們就可以在這個模組', 37.44, 39.28],
 ['撰寫任何需要用到的程式碼', 39.28, 42.04],
 ['舉例來說', 42.04, 43.14],
 ['這裡我建立一個簡單的函式', 43.14, 45.44],
 ['叫做打招呼', 45.44, 47.040000000000006],
 ['它可以讓別人輸入姓名', 47.040000000000006, 49.24],
 ['然後把這個名字寫成一句問候語', 49.24, 52.24],
 ['那這個函式目前只能在這個頁面中使用', 52.24, 55.84],
 ['如果我想要在另一個pycharm 文件', 55.84, 58.24],
 ['使用這個函式的話', 58.24, 59.6],
 ['就可以ollen', 59.6, 61.980000000000004]]

In [45]:
audio = AudioSegment.from_file("papaya_1.wav")
for i in range(len(sentence)):
    audio_clip = audio[sentence[i][1] *1000: sentence[i][2]*1000]
    audio_clip.export(f"sentences\\{(sentence[i][0].lower())}.wav", format="wav")

In [78]:
model = bundle.get_model()
model.to(device)

tokenizer = bundle.get_tokenizer()
aligner = bundle.get_aligner()

def compute_alignments(waveform: torch.Tensor, transcript: List[str]):
    with torch.inference_mode():
        emission, _ = model(waveform.to(device))
        token_spans = aligner(emission[0], tokenizer(transcript))
    return emission, token_spans

def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)


def plot_alignments(waveform, token_spans, emission, transcript, sample_rate=bundle.sample_rate):
    ratio = waveform.size(1) / emission.size(1) / sample_rate

    fig, axes = plt.subplots(2, 1)
    axes[0].imshow(emission[0].detach().cpu().T, aspect="auto")
    axes[0].set_title("Emission")
    axes[0].set_xticks([])

    axes[1].specgram(waveform[0], Fs=sample_rate)
    for t_spans, chars in zip(token_spans, transcript):
        t0, t1 = t_spans[0].start, t_spans[-1].end
        axes[0].axvspan(t0 - 0.5, t1 - 0.5, facecolor="None", hatch="/", edgecolor="white")
        axes[1].axvspan(ratio * t0, ratio * t1, facecolor="None", hatch="/", edgecolor="white")
        axes[1].annotate(f"{_score(t_spans):.2f}", (ratio * t0, sample_rate * 0.51), annotation_clip=False)

        for span, char in zip(t_spans, chars):
            t0 = span.start * ratio
            axes[1].annotate(char, (t0, sample_rate * 0.55), annotation_clip=False)

    axes[1].set_xlabel("time [second]")
    fig.tight_layout()

def preview_word(waveform, spans, num_frames, transcript, sample_rate):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    #print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    time_StarAndEnd = [ x0 / sample_rate, x1/sample_rate] # 回傳單個字的起始時間與結束時間
    segment = waveform[:, x0:x1]
    #return IPython.display.Audio(segment.numpy(), rate=sample_rate)
    return time_StarAndEnd
    

In [38]:
# with open('Rjay_1Fix.txt', 'r', encoding='utf-8') as file:
#     text_raw = file.read().lower()
#     file_name = file.name[:(file.name).find('.')]

# file.close()
# text_raw

'大家好我是fcgrjay本期為大家帶來的是大家期待已久的消防隊這個主題哦race救援王牌這卡組的話在目前的來說我覺得是非常非常的強哦就這個牌組剛出來的時候感覺他被稍微低估了一點但後續在一些小比賽或者那些金頭玩家開的練習房中race其實都不斷的有取得還不錯的成績那我自己覺得在天梯上使用起來race也算不錯的主題了所以這邊就推薦給大家那廢話不多說我們馬上開始介紹吧那這個卡組的話分為race軸然後還有篝火跟炎燐的這個軸然後和黑魔女軸那先講篝火和炎燐的軸還有黑魔女軸因為這個我們之前在蛇眼影片中都有提過所以我們就不再重述效果我們只講一下他在race這副卡組的應用首先是篝火跟炎燐部分篝火在這個卡組裡面只找炎燐因為我們沒有放藍色蛇眼所以我們唯一的檢索對象就是炎靈那為了避免炎燐跟篝火一起上手就是假設你只放一隻炎燐然後你同時抽到篝火加炎燐這種狀況發生那你手上篝火就不能開的關係所以我們就帶了兩隻炎燐但炎燐抽上手說實話對race來說也不是強卡因為這個牌組很吃通召的關係所以篝火開才是最好的方式那再來是魔女軸那魔女軸的話就是投入通緝令跟黑魔女那因為下個表通緝令限二的關係所以我們就帶兩個通緝令兩個黑魔女那黑魔女二到三都是ok的黑魔女和炎燐的檢索對象就是罪寶魔陷的話我們是帶了這個反叛然後還有原罪寶那先手的話基本上都是找原罪寶那後手的話才有機會是找反叛但大部分狀況下其實都還是優先調度原罪寶所以我們就看手牌反叛這張卡片的話是在ocg的race比較不帶的一張卡片哦那當然md的話因為你會遇到牌型種類很多你後手當你同時有篝火跟黑魔女的時候你會希望兩邊的牌效都打滿所以反叛就會變成一張可能要考慮帶的卡那再來是原罪寶原罪寶的話我們平常都是跳小藍或小綠那這邊是跳炎屬性等級一的關係所以我們會去調度消防栓這個卡組的黑魔女和這個炎燐都是擔任一個調度消防栓的角色哦那再來我們就進到這個卡組的主軸race就救援王牌本家的部分我們是帶了兩個消防栓然後三個這個四星然後兩個八星跟一個九星大哥那魔陷的話就是場地有帶然後這四個魔陷就是rescuealert和contain跟extinguish就各帶一張因為你沒有很希望他上手那最後是這個這一次新增加的emergency就三張那這個卡片的話非常非常強所以我們就直接帶滿那接下來快速為大家講解一下這個主題的效果首先我們看消防栓那消防栓一效果的話是場上有他以外的race怪獸的話對方就不可以指

In [115]:
# with open('papaya_1Fix.txt', 'r', encoding='utf-8') as file:
#     text_raw = file.read().lower()
#     file_name = file.name[:(file.name).find('.')]
# file.close()
# text_normalized = ' '.join(lazy_pinyin(text_raw))#將文字轉為沒有音調的拼音，lazy_pinyin是陣列所以要再join成字串

In [ ]:
# waveform, sample_rate = torchaudio.load('Rjay_1.mp3')
# waveform = waveform[0:1]

# transcript = text_normalized.split()
# emission, token_spans = compute_alignments(waveform, transcript)
# num_frames = emission.size(1)


# plot_alignments(waveform, token_spans, emission, transcript)

# print("Raw Transcript: ", text_raw)
# print("Normalized Transcript: ", text_normalized)
# IPython.display.Audio(waveform, rate=sample_rate)

In [84]:
for i in range(len(sentence)-1):#len(sentence)-1
    text_normalized = ' '.join(lazy_pinyin(sentence[i][0]))#將文字轉為沒有音調的拼音，lazy_pinyin是陣列所以要再join成字串

    waveform, sample_rate = torchaudio.load(f"sentences\\{sentence[i][0]}.wav")
    waveform = waveform[0:1]

    transcript = text_normalized.split()
    emission, token_spans = compute_alignments(waveform, transcript)
    num_frames = emission.size(1)


    #plot_alignments(waveform, token_spans, emission, transcript)

    print("Raw Transcript: ", sentence[i][0])
    print("Normalized Transcript: ", text_normalized)
    IPython.display.Audio(waveform, rate=sample_rate)

    text_raw = sentence[i][0]
    word_start_end = []
    pinyin_tone = pinyin(text_raw, style=Style.TONE3, heteronym=False)
    for j in range(len(transcript)):#len(transcript)
        timeStartEnd = preview_word(waveform, token_spans[j], num_frames, transcript[j], sample_rate)
        word_start_end.append([pinyin_tone[j][0], timeStartEnd[0], timeStartEnd[1]])
    print(word_start_end)

    audio = AudioSegment.from_file(f"sentences\\{sentence[i][0]}.wav")
    file_name = sentence[i][0]
    for k in range(len(word_start_end)):
        segment_audio = audio[word_start_end[k][1] *1000: word_start_end[k][2]*1000]
        segment_audio.export(f"data\\{file_name}-{k}_{word_start_end[k][0]}.wav", format="wav")
    print('------------------------------------------')

Raw Transcript:  hi 大家好
Normalized Transcript:  hi  da jia hao
[['hi ', 0.46820833333333334, 0.5016666666666667], ['da4', 0.7090208333333333, 0.7491458333333333], ['jia1', 0.8361041666666666, 0.8829375], ['hao3', 0.92975, 1.0836041666666667]]
------------------------------------------
Raw Transcript:  今天我們來聊聊 python 的 module 模組
Normalized Transcript:  jin tian wo men lai liao liao  python  de  module  mo zu
[['jin1', 0.06679166666666667, 0.167], ['tian1', 0.20708333333333334, 0.5544583333333334], ['wo3', 0.6613541666666667, 0.7147916666666667], ['men', 0.7348333333333333, 0.754875], ['lai2', 0.7816041666666667, 0.8483958333333333], ['liao2', 0.8483958333333333, 0.9552916666666667], ['liao2', 1.0020416666666667, 1.0888958333333334], [' python ', 1.1824166666666667, 1.4162291666666667], ['de', 1.4496458333333333, 1.5097708333333333], [' module ', 1.6634166666666668, 1.9373125], ['mo2', 1.9506666666666668, 2.1978541666666667], ['zu3', 2.3047291666666667, 2.384895833333333]]
--------------

In [68]:
timeStartEnd =preview_word(waveform, token_spans[0], num_frames, transcript[0], sample_rate) #sample rate使用跟輸入音檔儀降的就好了
timeStartEnd[0]
timeStartEnd[1]

hi (0.22): 0.468 - 0.502 sec


0.5016666666666667

In [69]:
text_raw = sentence[0][0]
word_start_end = []
pinyin_tone = pinyin(text_raw, style=Style.TONE3, heteronym=False)
for i in range(len(transcript)):#len(transcript)
    timeStartEnd = preview_word(waveform, token_spans[i], num_frames, transcript[i], sample_rate)
    word_start_end.append([pinyin_tone[i][0], timeStartEnd[0], timeStartEnd[1]])

hi (0.22): 0.468 - 0.502 sec
da (0.34): 0.709 - 0.749 sec
jia (0.21): 0.836 - 0.883 sec
hao (0.22): 0.930 - 1.084 sec


In [70]:
word_start_end

[['hi ', 0.46820833333333334, 0.5016666666666667],
 ['da4', 0.7090208333333333, 0.7491458333333333],
 ['jia1', 0.8361041666666666, 0.8829375],
 ['hao3', 0.92975, 1.0836041666666667]]

In [71]:
from pydub import AudioSegment
# 讀取音訊檔案
audio = AudioSegment.from_file(f"sentences\\{sentence[0][0]}.wav")

In [72]:
file_name = sentence[0][0]
for i in range(len(word_start_end)):
    segment_audio = audio[word_start_end[i][1] *1000: word_start_end[i][2]*1000]
    segment_audio.export(f"data\\{file_name}-{i}_{word_start_end[i][0]}.mp3", format="mp3")